In [2]:
conda activate reco_base


Note: you may need to restart the kernel to use updated packages.


In [3]:
cd D:\ADM Assignment 3\recommenders

D:\ADM Assignment 3\recommenders


In [4]:
import sys
sys.path.append("../../")
import os
import papermill as pm
import pandas as pd
import numpy as np
import tensorflow as tf
from reco_utils.common.timer import Timer
from reco_utils.recommender.deeprec.models.graphrec.lightgcn import LightGCN
from reco_utils.recommender.deeprec.DataModel.ImplicitCF import ImplicitCF
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_stratified_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.common.constants import SEED as DEFAULT_SEED
from reco_utils.recommender.deeprec.deeprec_utils import prepare_hparams


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.0.1
Tensorflow version: 1.15.3


In [16]:
# top k items to recommend
TOP_K = 10

# Select Snack data size: 100k, 1m, 10m, or 20m
SNACK_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 50
BATCH_SIZE = 1024

SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = "D:/ADM Assignment 3/recommenders/reco_utils/recommender/deeprec/config/lightgcn.yaml"
user_file = "D:/ADM Assignment 3/recommendersAssignment 3/user.csv"
item_file = "D:/ADM Assignment 3/recommendersAssignment 3/product.csv"

In [17]:
df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.81k/4.81k [00:03<00:00, 1.27kKB/s]


In [18]:
df.head()

,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [19]:
train, test = python_stratified_split(df, ratio=0.75)

In [20]:
data = ImplicitCF(train=train, test=test, seed=SEED)

In [21]:
hparams = prepare_hparams(yaml_file,
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [22]:
model = LightGCN(hparams, data, seed=SEED)

Already create adjacency matrix.


D:\ADM Assignment 3\recommenders\reco_utils\recommender\deeprec\DataModel\ImplicitCF.py:179: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -0.5).flatten()


Already normalize adjacency matrix.
Using xavier initialization.


In [23]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))

Epoch 1 (train)12.8s: train loss = 0.47261 = (mf)0.47237 + (embed)0.00024
Epoch 2 (train)10.9s: train loss = 0.31503 = (mf)0.31446 + (embed)0.00057
Epoch 3 (train)11.6s: train loss = 0.28676 = (mf)0.28609 + (embed)0.00067
Epoch 4 (train)10.9s: train loss = 0.24944 = (mf)0.24859 + (embed)0.00086
Epoch 5 (train)11.4s + (eval)0.7s: train loss = 0.23611 = (mf)0.23507 + (embed)0.00104, recall = 0.15527, ndcg = 0.34000, precision = 0.29629, map = 0.08896
Epoch 6 (train)10.0s: train loss = 0.22672 = (mf)0.22555 + (embed)0.00117
Epoch 7 (train)10.4s: train loss = 0.21514 = (mf)0.21384 + (embed)0.00130
Epoch 8 (train)11.6s: train loss = 0.20372 = (mf)0.20227 + (embed)0.00144
Epoch 9 (train)10.4s: train loss = 0.19039 = (mf)0.18879 + (embed)0.00160
Epoch 10 (train)10.0s + (eval)0.6s: train loss = 0.18599 = (mf)0.18422 + (embed)0.00177, recall = 0.17721, ndcg = 0.38189, precision = 0.33404, map = 0.10521
Epoch 11 (train)10.9s: train loss = 0.17536 = (mf)0.17344 + (embed)0.00192
Epoch 12 (train)11

In [24]:
topk_scores = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

topk_scores.head()

,userID,itemID,prediction
0,1,7,5.654733
1,1,475,5.286865
2,1,408,5.246353
3,1,919,5.183239
4,1,127,5.085067


In [25]:
eval_map = map_at_k(test, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(test, topk_scores, k=TOP_K)
eval_precision = precision_at_k(test, topk_scores, k=TOP_K)
eval_recall = recall_at_k(test, topk_scores, k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.135497
NDCG:	0.455034
Precision@K:	0.400530
Recall@K:	0.214454


In [17]:
model.infer_embedding(user_file, item_file)

In [1]:
cd D:\ADM Assignment 3\recommenders

D:\ADM Assignment 3\recommenders


In [18]:
import joblib

In [19]:
filename = 'LightGCN.sav'
joblib.dump(model, filename)

['LightGCN.sav']

In [26]:
topk_scores.to_csv("LightGCN.csv")